In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

path = "./Dataset"  # 데이터셋이 저장된 경로
file_list = os.listdir(path)    # 데이터셋의 목록 읽어오기

actions = []    # 파일 확장자 빼고 제스처 이름만 저장
for i in range(len(file_list)) :
    actions.append(file_list[i][:-4])

seq_length = 10

In [2]:
# 제스처별로 전처리하고 훈련데이터 시험데이터 생성하기
datax, datay = [], []

# for i in range(1):
for i in range(len(file_list)):

    # 데이터프레임 읽고 변수, 라벨 분리
    df = pd.read_csv(f'{path}/{file_list[i]}', header=None)
    dfx = df.iloc[:, :-1].to_numpy()
    dfy = df.iloc[:, -1].to_numpy()

    # 시퀀스길이에 따라 시퀀스데이터로 변환
    for i in range(len(df) - seq_length):
        datax.append(dfx[i: i + seq_length])
        datay.append(dfy[i + seq_length])

In [3]:
# # 정답지 원-핫 인코딩
# from keras.utils.np_utils import to_categorical
# datay = to_categorical(datay, num_classes=len(actions))

# 문제지 정답지 타입 변환
datax = np.array(datax)
datay = np.array(datay)

In [4]:
# 훈련데이터 시험데이터 분리
xtrain, xtest, ytrain, ytest = train_test_split(datax, datay, test_size=0.1)
print(xtrain.shape, xtest.shape, ytrain.shape, ytest.shape)

(342, 10, 15) (39, 10, 15) (342,) (39,)


In [6]:
from keras.models import Sequential
from keras import layers, callbacks

epochs=20

model6 = Sequential([
    layers.GRU(64, input_shape=xtrain.shape[1:], activation='tanh'),
    layers.Dense(32, activation='relu'),
    layers.Dense(len(actions), activation='softmax')
])

model6.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')

checkpoint = callbacks.ModelCheckpoint('./Models/model6.h5', save_best_only=True)
earlystopping = callbacks.EarlyStopping(patience=10, restore_best_weights=True)

hist6 = model6.fit(
                    xtrain, ytrain,
                    validation_split=0.2,
                    epochs = epochs,
                    callbacks = [checkpoint, earlystopping],
                    verbose=0
                )

score6 = model6.evaluate(xtest, ytest)

2/2 [==============================] - 0s 3ms/step - loss: 0.0030 - accuracy: 1.0000


In [11]:
from keras.models import load_model
from sklearn.metrics import multilabel_confusion_matrix

model = load_model('models/model.h5')

y_pred = model.predict(xtest)
y_pred

array([[4.7896359e-14, 1.0000000e+00, 2.7794795e-09],
       [1.0000000e+00, 3.7573428e-10, 7.5808976e-10],
       [9.9788874e-01, 2.1106414e-03, 5.5530961e-07],
       [1.0000000e+00, 2.6832971e-13, 3.8025065e-12],
       [1.8621324e-13, 1.0000000e+00, 3.8053849e-17],
       [1.0000000e+00, 6.8587565e-14, 5.3567686e-13],
       [3.0964719e-13, 1.0000000e+00, 2.9796721e-09],
       [1.0000000e+00, 1.9349954e-11, 8.5928548e-10],
       [4.3150265e-04, 9.9956399e-01, 4.5222509e-06],
       [1.3981425e-14, 1.0000000e+00, 1.1726040e-12],
       [9.9985266e-01, 1.6091659e-06, 1.4562621e-04],
       [5.1263105e-11, 1.0000000e+00, 3.6868820e-11],
       [1.5745471e-05, 1.1803895e-05, 9.9997246e-01],
       [5.3788135e-05, 1.5754291e-06, 9.9994469e-01],
       [1.0000000e+00, 8.9882979e-10, 1.2660593e-10],
       [1.0000000e+00, 2.1127698e-10, 2.1298310e-11],
       [8.1379167e-11, 7.0103874e-07, 9.9999928e-01],
       [4.2862524e-17, 1.0000000e+00, 8.6562883e-17],
       [1.0000000e+00, 7.073

In [15]:
ytest

array([1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 2, 2, 0, 0, 2, 1, 0, 2, 1, 0,
       1, 0, 1, 0, 1, 2, 2, 0, 1, 2, 0, 2, 1, 1, 0, 1, 1], dtype=int64)

In [16]:
# 원핫인코딩
from keras.utils.np_utils import to_categorical
ytestEn = to_categorical(ytest, num_classes=len(actions))
ytestEn

array([[0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [20]:
# multilabel_confusion_matrix(ytest, y_pred)
multilabel_confusion_matrix(np.argmax(ytestEn, axis=1), np.argmax(y_pred, axis=1))

array([[[24,  0],
        [ 0, 15]],

       [[23,  0],
        [ 0, 16]],

       [[31,  0],
        [ 0,  8]]], dtype=int64)